In [17]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset

import numpy as np

import re

import linecache

from tqdm import tqdm

In [2]:
device = torch.device("cpu")

In [195]:
class SimpleGRU(nn.Module):
    def __init__(self, hidden_size, embedding_path, vocab_path, num_layers=4):
        super(SimpleGRU, self).__init__()

        self.num_layers = num_layers
        self.hidden_size = hidden_size

        word_embeddings = np.load(embedding_path)
        self.vocab = np.load(vocab_path)

        self.embedding_layer = nn.Embedding.from_pretrained(torch.from_numpy(word_embeddings).float())
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=num_layers)

        self.pool = nn.AdaptiveMaxPool1d(1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x, hidden):
        embeddings = self.embedding_layer(x)

        final_output = torch.zeros_like(embeddings)
        
        for i in range(embeddings.shape[1]):
            output, hidden = self.gru(embeddings[:,i,:].view(1, 1, -1), hidden)
            final_output[:,i,:] = output
        
        final_output = self.pool(final_output)
        final_output = self.sigmoid(final_output)

        return final_output.squeeze(2)


    def init_hidden(self):
        return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)


    def get_indices(self, string):
        indices = []

        string_arr = string.split(" ")

        for word in string_arr:
            if np.where(self.vocab == word)[0].shape[0] != 0:
                indices += [np.where(self.vocab == word)[0]]
            else:
                indices += [np.array([0])]
        
        return torch.tensor(indices).squeeze(1)

In [234]:
class EntityDataset(Dataset):
    # overriden methods
    def __init__(self, file_path, model):
        self.file_path = file_path

        self.model = model


    def __len__(self):
        with open(self.file_path, "rbU") as f:
            num_lines = sum(1 for _ in f)
        
        # don't count first line
        return num_lines - 1


    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        particular_line = linecache.getline(self.file_path, idx+1)
        cleaned_sample = self.clean_up(particular_line)

        input_sentence = cleaned_sample[4]
        entity1 = cleaned_sample[1]
        entity2 = cleaned_sample[2]

        sentence_arr = self.model.get_indices(input_sentence)
        entity1_arr = self.model.get_indices(entity1)
        entity2_arr = self.model.get_indices(entity2)

        '''labels = torch.zeros_like(sentence_arr)

        for _, word_index in enumerate(sentence_arr):
            for entity_index in entity1_arr:
                if entity_index == word_index:
                    labels[_] = 1
            for entity_index in entity2_arr:
                if entity_index == word_index:
                    labels[_] = 1'''

        labels1 = self.generate_labels(sentence_arr, entity1_arr)
        labels2 = self.generate_labels(sentence_arr, entity2_arr)

        labels = labels1 | labels2

        return sentence_arr, labels.type(torch.float32)
    

    # first instance of entity in sentence
    def generate_labels(self, sentence_arr, entity_arr):
        correct = []

        for _, token in enumerate(sentence_arr):
            if _ < sentence_arr.shape[0] - entity_arr.shape[0]:
                if token == entity_arr[0]:
                    not_equal = False
                    for i, val in enumerate(entity_arr):
                        if sentence_arr[_+i] != val:
                            not_equal = True
                    if not not_equal:
                        correct += [_]
            else:
                break
        
        labels = torch.zeros_like(sentence_arr)
        for i in correct:
            labels[i:i+entity_arr.shape[0]] = 1
        
        return labels
        

    # helper
    def clean_up(self, line):
        remove_chars = ["<e1>", "</e1>", "<e2>", "</e2>"]

        line = line.strip()

        for char in remove_chars:
            line = line.replace(char, "")
        
        # string clean up
        line = re.sub(r'[^a-zA-Z1-9\s]', '', line)
        line = re.sub(' +', ' ', line)
        line = line.lower()

        line_data = line.split("\t")
        
        return line_data

In [235]:
test_string = "hello how are you"

model = SimpleGRU(50, "utils/embs_npa.npy", "utils/vocab_npa.npy")

indices = model.get_indices(test_string)

print(indices.shape)

start_hidden = model.init_hidden()
output = model(indices.unsqueeze(0), start_hidden)

print(output.shape)

torch.Size([4])
torch.Size([1, 4])


In [236]:
batch_size = 1

dataset = EntityDataset('data/en_corpora_test.txt', model)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

/tmp/ipykernel_11013/1136066156.py:10: DeprecationWarning: 'U' mode is deprecated
  with open(self.file_path, "rbU") as f:


In [237]:
sample = next(iter(train_loader))

print(sample)

/tmp/ipykernel_11013/1136066156.py:10: DeprecationWarning: 'U' mode is deprecated
  with open(self.file_path, "rbU") as f:


[tensor([[189584,     17,     31,   2490,   1139,    777,   1349,      8,  48947,
            747,      8,    949,      2,    777,   5711,      5,   1486,  13356,
           6639,   3824,    281,  44075,   7302,      7,    686,      0,   4684,
           6851,   6639]]), tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.]])]


In [238]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [239]:
EPOCHS = 10

for epoch in range(EPOCHS):
    for _, sample in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()

        input = sample[0]
        labels = sample[1]

        start_hidden = model.init_hidden()
        output = model(input, start_hidden)

        loss = criterion(output, labels)

        loss.backward()

        optimizer.step()

        if _ % 50 == 0:
            print(loss.item())

/tmp/ipykernel_11013/1136066156.py:10: DeprecationWarning: 'U' mode is deprecated
  with open(self.file_path, "rbU") as f:
  0%|          | 1/5461 [00:00<27:15,  3.34it/s]

0.8746582269668579


  1%|          | 51/5461 [00:15<24:24,  3.69it/s]

0.7528480887413025


  2%|▏         | 101/5461 [00:31<26:05,  3.42it/s]

...

 23%|██▎       | 1251/5461 [06:33<21:20,  3.29it/s]

0.4940255284309387


 24%|██▍       | 1301/5461 [06:48<21:41,  3.20it/s]

0.5244684815406799


 24%|██▍       | 1321/5461 [06:54<19:08,  3.61it/s]